# Notebook to Diagram

While the drag-and-drop behavior of MIME outputs works well, it is untenable at scale, and doesn't handle some features very well.

> # A Diagram Notebook

> There _might_ be a compelling reason to store a drawio diagram in a notebook. This has one.

In [ ]:
import pandas
%config InlineBackend.figure_formats = ['svg']

In [ ]:
pandas.util.testing.makeDataFrame().plot();

In [ ]:
from pathlib import Path
import requests
import copy
import os
import html
import difflib
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter
from lxml.builder import E
from uuid import uuid4
import IPython
import bleach
import black

from nbconvert.exporters.html import TemplateExporter
from nbconvert.filters import markdown2html_mistune
from jinja2 import DictLoader

In [ ]:
from ipydrawio import Diagram

## What XML Library?

`xml.etree` is part of the standard library, but `lxml.etree` is faster at scale. Let's prefer the latter, and offers some nice API improvements.

In [ ]:
import lxml.etree as ET

# A Notebook

Using this notebook is as good as any.

In [ ]:
import nbformat
this_notebook = nbformat.reads(Path(os.environ.get("NOTEBOOK", "Notebook to Diagram.ipynb")).read_text(encoding="utf-8"), as_version=4)
this_notebook.cells[0]

## Exporter

We want to tie into the `nbconvert` pipeline pretty directly. We'll do it in-line for now.

In [ ]:
from ipydrawio.constants import A_SHORT_DRAWIO
A_SHORT_DRAWIO

In [ ]:
def a_style(**kwargs): return dict(style="".join([f"{k}={v};" for k, v in kwargs.items()]))

In [ ]:
wikimedia = "https://tools-static.wmflabs.org/fontcdn/css?family=Architects+Daughter"

In [ ]:
def a_note_style(**kwargs):
    _defaults_ = kwargs.pop("_defaults_", dict(
        shape="note",
        backgroundOutline=1,
        darkOpacity=0.05,
        fillColor="#FFF9B2",
        strokeColor="none",
        fillStyle="solid",
        direction="west",
        gradientDirection="north",
        gradientColor="#FFF2A1",
        sketch=1,
        shadow=1,
        size=20,
        fontSize=24,
        jiggle=2,
        pointerEvents=1,
        hachureGap=4,
        whiteSpace="wrap",
        fontFamily="Architects Daughter",
        fontSource="https%3A%2F%2Ftools-static.wmflabs.org%2Ffontcdn%2Fcss%3Ffamily%3DArchitects%2BDaughter"
    ))
    style = dict(**_defaults_)
    style.update(**kwargs)
    return a_style(**style)

In [ ]:
def a_note_geometry(**kwargs):
    _defaults_ = kwargs.get("_defaults_", dict(
        x="0", y="0", width="500", height="300", **{"as": "geometry"}
    ))
    mx_kwargs = dict(_defaults_)
    mx_kwargs.pop("defaults", None)
    mx_kwargs.update(**kwargs)
    return E.mxGeometry(**mx_kwargs)

In [ ]:
def a_note(value=None, geometry=None, vertex="1", parent="1", **kwargs):
    value = value or kwargs.pop("value", None)
    _defaults_ = kwargs.pop("_defaults_", dict(
        id=f"{uuid4()}",
        value=value,
    ))
    mx_kwargs = dict(_defaults_)
    mx_kwargs.update(**kwargs)
    style = mx_kwargs.pop("style", {})
    if isinstance(style, dict):
        mx_kwargs.update(a_note_style(**style))
    mx_kwargs.update(
        vertex=kwargs.get("vertex", "1"),
        parent=kwargs.get("parent", "1"),
    )
    geometry = a_note_geometry(**(geometry or {}))
    return E.mxCell(geometry, **mx_kwargs)

In [ ]:
A_CARD = ("""
<mxCell id="64" value="CARD A" style="shape=note;backgroundOutline=1;darkOpacity=0.05;fillColor=#FFF9B2;strokeColor=none;fillStyle=solid;direction=west;gradientDirection=north;gradientColor=#FFF2A1;sketch=1;shadow=1;size=20;fontSize=24;jiggle=2;pointerEvents=1;hachureGap=4;whiteSpace=wrap;fontFamily=Architects Daughter;fontSource=https%3A%2F%2Ftools-static.wmflabs.org%2Ffontcdn%2Fcss%3Ffamily%3DArchitects%2BDaughter;" vertex="1" parent="1">
  <mxGeometry x="0" y="0" width="500" height="300" as="geometry"/>
</mxCell>""").strip()

In [ ]:
def test_a_note():
    note = a_note(value="CARD A", id="64")
    xml = ET.tostring(note, encoding=str, pretty_print=True)
    lines = [xml.splitlines(), A_CARD.splitlines()]
    diff = "\n".join([*difflib.unified_diff(*lines)])
    if diff:
        htmldiff = difflib.HtmlDiff()
        IPython.display.display(IPython.display.HTML(htmldiff.make_file(*lines)))
    assert not diff, diff

In [ ]:
test_a_note()

In [ ]:
ET.tostring(a_note("A Note"))

In [ ]:
class MXTools:
    page_height = 850
    page_width = 1100
    
    def empty_mx(self):
        return E.mxfile(
            E.diagram(
                E.mxGraphModel(
                    E.root(
                        E.mxCell(id="0"),
                        E.mxCell(id="1", parent="0"),
                    ),
                    dx="1450",
                    dy="467",
                    grid="1",
                    gridSize="10",
                    guides="1",
                    tooltips="1",
                    connect="1",
                    arrows="1",
                    fold="1",
                    page="1",
                    pageScale="1",
                    pageWidth=f"{self.page_height}",
                    pageHeight=f"{self.page_width}",
                    math="0",
                    shadow="0",
                ),
                id="x", 
                name="Page-1"
            ),
            version="14.6.11",
        )

    def style(self, **kwargs):
        return dict(style=";".join([f"{k}={v}" for k, v in kwargs.items()]) + ";")
    
    @property
    def text_style(self):
        return self.style(
            rounded=0,
            verticalAlign="top",
            autosize=1,
            align="left",
            whiteSpace="wrap",
            fillColor="black",
            strokeColor="none",
            html="1"
        )
    
    def svg_style(self, svg):
        return self.style(
            shape="image",
            verticalLabelPosition="bottom",
            labelBackgroundColor="#ffffff",
            verticalAlign="top",
            aspect="fixed",
            imageAspect=0,
            image=f"data:image/svg+xml,{svg}"
        )

In [ ]:
DEFAULT_FORMATTER = HtmlFormatter(
    noclasses=True, 
    style="monokai", 
    wrapcode=True, 
    cssstyles='background: rgba(0,0,0,0.9); padding: 0.1em 0.5em; border-radius: 0.5em'
)

In [ ]:
class NoteCellExporter(TemplateExporter, MXTools):
    export_from_notebook = "Diagram"
    # todo: traitliet
    formatter = DEFAULT_FORMATTER
    lexer = get_lexer_by_name("ipython")

    def _file_extension_default(self):
        """
        The new file extension is ``.test_ext``
        """
        return '.dio'

    def from_notebook_node(self, nb, resources=None, **kw):
        nb_copy = copy.deepcopy(nb)
        resources = self._init_resources(resources)

        if 'language' in nb['metadata']:
            resources['language'] = nb['metadata']['language'].lower()

        # Preprocess
        nb_copy, resources = self._preprocess(nb_copy, resources)

        tree = self.empty_mx()
        root = tree.xpath("//root")
        prev = tree.xpath("//mxCell")[-1]
        prev_id = 1
        col = 0
        row = 0
        w = 500
        h = 300
        p = 25
        card_geo = lambda: dict(
            x=f"{col * (w + p)}", y=f"{row * (h + p)}", 
            width=f"{w}", height=f"{h}",
            **{"as": "geometry"}
        )
    
        for cell in nb_copy["cells"]:
            if cell["cell_type"] == "markdown":
                row += 1
                col = 0
                mxc = a_note(
                    value=self.markdown_escaped(cell),
                    id=f"{prev_id + 1}",
                    style=dict(
                        html="1",
                        fontFamily="Architects Daughter",
                        fontSource="https://tools-static.wmflabs.org/fontcdn/css?family=Architects+Daughter",
                    ),
                    geometry=dict(y=f"{row * (h + p)}", x=f"{col * (w + p)}")
                )
                prev.addnext(mxc)
                prev = mxc
                prev_id += 1
                col += 1
            elif cell["cell_type"] == "code":
                mxc = E.mxCell(
                    a_note_geometry(**card_geo()),
                    id=f"{prev_id + 1}", parent="1", vertex="1",
                    value=self.source_escaped(cell),
                    **self.text_style
                )
                prev.addnext(mxc)
                prev = mxc
                prev_id += 1
                col += 1
                for output in cell.outputs:
                    if "data" not in output:
                        continue
                    svg = output.data.get("image/svg+xml")
                    if svg:
                        mxc = E.mxCell(
                            a_note_geometry(**card_geo()),
                            id=f"{prev_id + 1}", parent="1", vertex="1",
                            value="",
                            **self.svg_style(self.svg_escaped(svg))
                        )
                        prev.addnext(mxc)
                        prev = mxc
                        prev_id += 1
                        col += 1
        return ET.tostring(tree, encoding=str, pretty_print=True).replace("&amp;", "&"), resources


    def source_escaped(self, cell):
        try:
            source = black.format_str(cell.source, mode=black.FileMode(line_length=60))
        except:
            source = cell.source
        return html.escape(
            highlight(
                source,
                self.lexer,
                self.formatter
            )
        )

    allowed_tags = {
        *bleach.sanitizer.ALLOWED_TAGS,
        *{f"h{i}" for i in range(7)},
        "div", "pre", "span", "blockquote", "strong"
    } ^ {"a"}

    def markdown_escaped(self, cell):
        return html.escape(
            bleach.clean(
                markdown2html_mistune(cell.source),
                tags=self.allowed_tags,
                strip=True
            )
        )

    def svg_escaped(self, some_data):
        return requests.utils.quote(some_data, safe="!*()")

    def svg_size(self, some_data):
        width = 100
        height = 100
        et = ET.fromstring(some_data.encode("utf-8"))
        width = et.xpath("/@width")
        height = et.xpath("/@height")
        return f"""width="{width}" height="{height}" """

In [ ]:
export_diagram = NoteCellExporter()
(body, resources) = export_diagram.from_notebook_node(this_notebook)
# print(body)

In [ ]:
diagram = Diagram(layout=dict(height="400px", width="100%"))
new_params = dict(**diagram.url_params)
new_params.update(ui="sketch", format="0")
diagram.url_params = new_params
diagram

In [ ]:
diagram.source.value = body

In [ ]:
print(ET.tostring(ET.fromstring(diagram.source.value), encoding=str, pretty_print=True))

## Packaging

Not appearing here, but our `setup.cfg` should be upgraded to something like:


```ini
[options.entry_points]
nbconvert.exporters =
    ipydrawio = ipydrawio.exporters:DiagramExporter
```